## Trying BEGAN in Faces image

https://www.reddit.com/r/MachineLearning/comments/633jal/r170310717_began_boundary_equilibrium_generative/

https://arxiv.org/pdf/1703.10717.pdf

1. The generator wins by generating images that the discriminator can successfully autoencode with small loss.

2. The discriminator wins by autoencoding real images well and by autoencoding the generated images poorly.


Auto-Encoder

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/autoencoder.ipynb

awesome python library
https://pypi.python.org/pypi/tqdm

https://github.com/carpedm20/BEGAN-tensorflow/blob/master/models.py

https://arxiv.org/pdf/1703.10717.pdf


### Library and Utils / Helper functions

In [1]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import re
import scipy.misc
import scipy
from functools import partial
from collections import OrderedDict
import glob
import traceback

In [2]:
def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        path_dir, _ = os.path.split(path)
        if not os.path.isdir(path_dir):
            os.makedirs(path_dir)


def session(graph=None, allow_soft_placement=True,
            log_device_placement=False, allow_growth=True):
    """ return a Session with simple config """

    config = tf.ConfigProto(allow_soft_placement=allow_soft_placement,
                            log_device_placement=log_device_placement)
    config.gpu_options.allow_growth = allow_growth
    return tf.Session(graph=graph, config=config)


def tensors_filter(tensors, filters, combine_type='or'):
    assert isinstance(tensors, (list, tuple)), '`tensors` shoule be a list or tuple!'
    assert isinstance(filters, (str, list, tuple)), \
        '`filters` should be a string or a list(tuple) of strings!'
    assert combine_type == 'or' or combine_type == 'and', "`combine_type` should be 'or' or 'and'!"

    if isinstance(filters, str):
        filters = [filters]

    f_tens = []
    for ten in tensors:
        if combine_type == 'or':
            for filt in filters:
                if filt in ten.name:
                    f_tens.append(ten)
                    break
        elif combine_type == 'and':
            all_pass = True
            for filt in filters:
                if filt not in ten.name:
                    all_pass = False
                    break
            if all_pass:
                f_tens.append(ten)
    return f_tens


def trainable_variables(filters=None, combine_type='or'):
    t_var = tf.trainable_variables()
    if filters is None:
        return t_var
    else:
        return tensors_filter(t_var, filters, combine_type)


def summary(tensor_collection, summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram']):
    """
    usage:
    1. summary(tensor)
    2. summary([tensor_a, tensor_b])
    3. summary({tensor_a: 'a', tensor_b: 'b})
    """

    def _summary(tensor, name, summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram']):
        """ Attach a lot of summaries to a Tensor. """

        if name is None:
            # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
            # session. This helps the clarity of presentation on tensorboard.
            name = re.sub('%s_[0-9]*/' % 'tower', '', tensor.name)
            name = re.sub(':', '-', name)

        with tf.name_scope('summary_' + name):
            summaries = []
            if len(tensor._shape) == 0:
                summaries.append(tf.summary.scalar(name, tensor))
            else:
                if 'mean' in summary_type:
                    mean = tf.reduce_mean(tensor)
                    summaries.append(tf.summary.scalar(name + '/mean', mean))
                if 'stddev' in summary_type:
                    mean = tf.reduce_mean(tensor)
                    stddev = tf.sqrt(tf.reduce_mean(tf.square(tensor - mean)))
                    summaries.append(tf.summary.scalar(name + '/stddev', stddev))
                if 'max' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/max', tf.reduce_max(tensor)))
                if 'min' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/min', tf.reduce_min(tensor)))
                if 'sparsity' in summary_type:
                    summaries.append(tf.summary.scalar(name + '/sparsity', tf.nn.zero_fraction(tensor)))
                if 'histogram' in summary_type:
                    summaries.append(tf.summary.histogram(name, tensor))
            return tf.summary.merge(summaries)

    if not isinstance(tensor_collection, (list, tuple, dict)):
        tensor_collection = [tensor_collection]
    with tf.name_scope('summaries'):
        summaries = []
        if isinstance(tensor_collection, (list, tuple)):
            for tensor in tensor_collection:
                summaries.append(_summary(tensor, None, summary_type))
        else:
            for tensor, name in tensor_collection.items():
                summaries.append(_summary(tensor, name, summary_type))
        return tf.summary.merge(summaries)



def load_checkpoint(checkpoint_dir, session, var_list=None):
    print(' [*] Loading checkpoint...')
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        ckpt_path = os.path.join(checkpoint_dir, ckpt_name)
    try:
        restorer = tf.train.Saver(var_list)
        restorer.restore(session, ckpt_path)
        print(' [*] Loading successful! Copy variables from % s' % ckpt_path)
        return True
    except:
        print(' [*] No suitable checkpoint!')
        return False



def disk_image_batch(image_paths, batch_size, shape, preprocess_fn=None, shuffle=True, num_threads=16,
                     min_after_dequeue=100, allow_smaller_final_batch=False, scope=None):
    """
    This function is suitable for bmp, jpg, png and gif files
    image_paths: string list or 1-D tensor, each of which is an iamge path
    preprocess_fn: single image preprocessing function
    """

    with tf.name_scope(scope, 'disk_image_batch'):
        data_num = len(image_paths)

        # dequeue a single image path and read the image bytes; enqueue the whole file list
        _, img = tf.WholeFileReader().read(tf.train.string_input_producer(image_paths, shuffle=shuffle, capacity=data_num))
        img = tf.image.decode_image(img)

        # preprocessing
        img.set_shape(shape)
        if preprocess_fn is not None:
            img = preprocess_fn(img)

        # batch datas
        if shuffle:
            capacity = min_after_dequeue + (num_threads + 1) * batch_size
            img_batch = tf.train.shuffle_batch([img],
                                               batch_size=batch_size,
                                               capacity=capacity,
                                               min_after_dequeue=min_after_dequeue,
                                               num_threads=num_threads,
                                               allow_smaller_final_batch=allow_smaller_final_batch)
        else:
            img_batch = tf.train.batch([img],
                                       batch_size=batch_size,
                                       allow_smaller_final_batch=allow_smaller_final_batch)

        return img_batch, data_num


class DiskImageData:

    def __init__(self, image_paths, batch_size, shape, preprocess_fn=None, shuffle=True, num_threads=16,
                 min_after_dequeue=100, allow_smaller_final_batch=False, scope=None):
        """
        This function is suitable for bmp, jpg, png and gif files
        image_paths: string list or 1-D tensor, each of which is an iamge path
        preprocess_fn: single image preprocessing function
        """

        self.graph = tf.Graph()  # declare ops in a separated graph
        with self.graph.as_default():
            # @TODO
            # There are some strange errors if the gpu device is the
            # same with the main graph, but cpu device is ok. I don't know why...
            with tf.device('/cpu:0'):
                self._batch_ops, self._data_num = disk_image_batch(image_paths, batch_size, shape, preprocess_fn, shuffle, num_threads,
                                                                   min_after_dequeue, allow_smaller_final_batch, scope)

        print(' [*] DiskImageData: create session!')
        self.sess = session(graph=self.graph)
        self.coord = tf.train.Coordinator()
        self.threads = tf.train.start_queue_runners(sess=self.sess, coord=self.coord)

    def __len__(self):
        return self._data_num

    def batch(self):
        return self.sess.run(self._batch_ops)

    def __del__(self):
        print(' [*] DiskImageData: stop threads and close session!')
        self.coord.request_stop()
        self.coord.join(self.threads)
        self.sess.close()


def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """
    transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    """
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def imwrite(image, path):
    """ save an [-1.0, 1.0] image """

    if image.ndim == 3 and image.shape[2] == 1:  # for gray image
        image = np.array(image, copy=True)
        image.shape = image.shape[0:2]
    return scipy.misc.imsave(path, to_range(image, 0, 255, np.uint8))


def immerge(images, row, col):
    """
    merge images into an image with (row * h) * (col * w)
    `images` is in shape of N * H * W(* C=1 or 3)
    """

    h, w = images.shape[1], images.shape[2]
    if images.ndim == 4:
        img = np.zeros((h * row, w * col, images.shape[3]))
    elif images.ndim == 3:
        img = np.zeros((h * row, w * col))
    for idx, image in enumerate(images):
        i = idx % col
        j = idx // col
        img[j * h:j * h + h, i * w:i * w + w, ...] = image

    return img

In [3]:
def flatten_fully_connected(inputs,
                            num_outputs,
                            activation_fn=tf.nn.relu,
                            normalizer_fn=None,
                            normalizer_params=None,
                            weights_initializer=slim.xavier_initializer(),
                            weights_regularizer=None,
                            biases_initializer=tf.zeros_initializer(),
                            biases_regularizer=None,
                            reuse=None,
                            variables_collections=None,
                            outputs_collections=None,
                            trainable=True,
                            scope=None):
    with tf.variable_scope(scope, 'flatten_fully_connected', [inputs]):
        if inputs.shape.ndims > 2:
            inputs = slim.flatten(inputs)
        return slim.fully_connected(inputs,
                                    num_outputs,
                                    activation_fn,
                                    normalizer_fn,
                                    normalizer_params,
                                    weights_initializer,
                                    weights_regularizer,
                                    biases_initializer,
                                    biases_regularizer,
                                    reuse,
                                    variables_collections,
                                    outputs_collections,
                                    trainable,
                                    scope)


def leak_relu(x, leak, scope=None):
    with tf.name_scope(scope, 'leak_relu', [x, leak]):
        if leak < 1:
            y = tf.maximum(x, leak * x)
        else:
            y = tf.minimum(x, leak * x)
        return y
    
def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))

def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

### Layer wrappers

In [4]:
'''
slim layers:

    conv2d(args):
        inputs: 
            A 4-D tensor with dimensions [batch_size, height, width, channels]
        num_outputs:
            Integer, the number of output filters.
        kernel_size:
            Can be an int if both values are the same.
        stride: 
            default=1
        ...
    
    conv2d_transpose(args):
        inputs:
        num_outputs: 
            Integer, the number of output filters.
        kernel_size:
        stride: 
            default=1
        ...
        
    batch_norm(args):
        is_training: 
            Whether or not the layer is in training mode. In training mode
            it would accumulate the statistics of the moments into `moving_mean` and
            `moving_variance` using an exponential moving average with the given
            `decay`. When it is not in training mode then it would use the values of
            the `moving_mean` and the `moving_variance`.
            
'''

# conv = partial(slim.conv2d, activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev=0.02))
# dconv = partial(slim.conv2d_transpose, activation_fn=None, weights_initializer=tf.random_normal_initializer(stddev=0.02))
# fc = partial(flatten_fully_connected, activation_fn=None, weights_initializer=tf.random_normal_initializer(stddev=0.02))

conv = partial(slim.conv2d, activation_fn=None)
dconv = partial(slim.conv2d_transpose, activation_fn=None)
fc = partial(flatten_fully_connected, activation_fn=None)


relu = tf.nn.relu
elu = tf.nn.elu
lrelu = partial(leak_relu, leak=0.2)
batch_norm = partial(slim.batch_norm, decay=0.9, scale=True, epsilon=1e-5, updates_collections=None)
ln = slim.layer_norm

In [5]:
def generator(z, data_size=64, filter_num=64, reuse=True, training=True):
    bn = partial(batch_norm, is_training=training)
    conv_bn_elu = partial(conv, normalizer_fn=bn, activation_fn=elu, biases_initializer=None)

    with tf.variable_scope('generator', reuse=reuse):
        y = fc(z, 8 * 8 * filter_num)
        y = tf.reshape(y, [-1, 8, 8, filter_num])
        
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        
        y = resize_nn(y, data_size / 4)
        
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        
        y = resize_nn(y, data_size / 2)
        
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        
        y = resize_nn(y, data_size)
        
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        y = conv_bn_elu(y, num_outputs=filter_num, kernel_size=3, stride=1)
        
        img = tf.tanh(conv(y, num_outputs=3, kernel_size=3, stride=1)) # Change number of out channels
        return img


def discriminator(img, data_size=64, filter_num=64, reuse=True, training=True):
    bn = partial(batch_norm, is_training=training)
    conv_bn_elu = partial(conv, normalizer_fn=bn, activation_fn=elu, biases_initializer=None)
    h = 64 # or 128
    
    with tf.variable_scope('discriminator', reuse=reuse):
        encoder = conv_bn_elu(img, num_outputs=filter_num, kernel_size=3, stride=1)
        
        encoder = conv_bn_elu(encoder, num_outputs=filter_num, kernel_size=3, stride=1)
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 2, kernel_size=3, stride=2)
        
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 2, kernel_size=3, stride=1)
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 3, kernel_size=3, stride=2)
        
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 3, kernel_size=3, stride=1)
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 4, kernel_size=3, stride=2)
        
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 4, kernel_size=3, stride=1)
        encoder = conv_bn_elu(encoder, num_outputs=filter_num * 4, kernel_size=3, stride=1)

        latent_space = fc(encoder, h)
        
        decoder = fc(latent_space, 8 * 8 * filter_num)
        decoder = tf.reshape(decoder, [-1, 8, 8, filter_num])
        
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        
        decoder = resize_nn(decoder, data_size / 4)
        
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        
        decoder = resize_nn(decoder, data_size / 2)
        
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        
        decoder = resize_nn(decoder, data_size)
        
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        decoder = conv_bn_elu(decoder, num_outputs=filter_num, kernel_size=3, stride=1)
        
        reconst = tf.tanh(conv(decoder, num_outputs=3, kernel_size=3, stride=1))
        return reconst

### Connecting them together

In [6]:
""" param """
epoch = 50000
batch_size = 32 # 16 from paper
lr_in = 0.0001
z_dim = 64
clip = 0.01
n_critic = 5
gpu_id = 0

lamda = 0.001
gamma = 0.5
beta1 = 0.5 #0.9

# The number of run
run_num = 10

''' data '''


def preprocess_fn(img):
    #Transform it to be between -1 and 1
    img = tf.to_float(img) / 127.5 - 1

    return img

file_paths = glob.glob("./Face_data/Celeba/dataset_64x64/*")
data_pool = DiskImageData(file_paths, batch_size, shape=[64, 64, 3], preprocess_fn=preprocess_fn)


""" graphs """
with tf.device('/gpu:%d' % gpu_id):

    ''' graph '''
    # inputs
    real = tf.placeholder(tf.float32, shape=[None, 64, 64, 3])
    z = tf.placeholder(tf.float32, shape=[None, z_dim])
    k_t = tf.placeholder(tf.float32, shape=(), name='kt') # shape of a scalar = ()
    lr = tf.placeholder(tf.float32, name='lr')

    # generate
    fake = generator(z, reuse=False)

    # discriminate
    reconst_r = discriminator(real, reuse=False)
    reconst_f = discriminator(fake)

    # losses
    AE_loss_r = l1_loss(real, reconst_r)
    AE_loss_f = l1_loss(fake, reconst_f)
    d_loss = AE_loss_r - k_t * AE_loss_f
    g_loss = AE_loss_f
    M_global = AE_loss_r + tf.abs(gamma * AE_loss_r - AE_loss_f)

    # vars
    d_vars = trainable_variables('discriminator')
    g_vars = trainable_variables('generator')
    
    # optimizers
    d_opt = tf.train.AdamOptimizer(lr, beta1).minimize(d_loss, var_list=d_vars)
    g_opt = tf.train.AdamOptimizer(lr, beta1).minimize(g_loss, var_list=g_vars)
    
    # summaries
    d_summary = summary({AE_loss_r: 'AE_real_reconst_loss', d_loss: 'd_loss'})
    g_summary = summary({g_loss: 'g_loss'})
    m_summary = summary({M_global: 'M_global'})

    # sample
    f_sample = generator(z, training=False)


 [*] DiskImageData: create session!


### Training the network

In [7]:
""" train """
''' init '''
# session
sess = session()
# saver
saver = tf.train.Saver(max_to_keep=5)
# summary writer
summary_writer = tf.summary.FileWriter('./BEGAN/summaries/run%s' % run_num, sess.graph)

''' initialization '''
ckpt_dir = './BEGAN/checkpoints/run%s' % run_num
mkdir(ckpt_dir + '/')
if not load_checkpoint(ckpt_dir, sess):
    sess.run(tf.global_variables_initializer())

''' train '''
try:
    #z_ipt_sample = np.random.normal(size=[100, z_dim])
    z_ipt_sample = np.random.uniform(-1., 1., size=[100, z_dim])
    kt = np.float32(0.)
    lr_i = np.float32(lr_in)
    
    batch_epoch = len(data_pool) // (batch_size * n_critic)
    max_it = epoch * batch_epoch

    for it in range(max_it):

        # which epoch
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # batch data
        real_ipt = data_pool.batch()
        
        z_ipt = np.random.uniform(-1., 1., size=[batch_size, z_dim])

        # terms to be calculated & feed list
        g_opt_list = [g_opt, g_loss, AE_loss_r, AE_loss_f]
        d_opt_list = [d_opt, d_loss, d_summary, g_summary, m_summary]
        feed_dict = {real: real_ipt, z: z_ipt, k_t: kt, lr: lr_i}

        # run
        _, g_loss_calculated, AE_loss_r_calculated, AE_loss_f_calculated = sess.run(g_opt_list, feed_dict=feed_dict)
        _, d_loss_calculated, summary_d, summary_g, summary_m = sess.run(d_opt_list, feed_dict=feed_dict)
        
        # update kt, m_global -- (The range of kt is [0,1])
        kt = kt + lamda * (gamma * AE_loss_r_calculated - AE_loss_f_calculated)
        kt = np.maximum(np.minimum(1.,kt), 0.)
        m_global = AE_loss_r_calculated + np.abs(gamma * AE_loss_r_calculated - AE_loss_f_calculated)
        
        # write train summary
        summary_writer.add_summary(summary_d, it)
        summary_writer.add_summary(summary_g, it)
        summary_writer.add_summary(summary_m, it)

        

        # display
        if it % 100 == 0:
            print("Epoch: (%3d) (%5d/%5d) -- g_loss: %.4f, d_loss: %.4f AE_r: %.4f, AE_f: %.4f, kt: %.8f, M: %.8f" 
                  % (epoch, it_epoch, batch_epoch, 
                     g_loss_calculated, d_loss_calculated,
                     AE_loss_r_calculated, AE_loss_f_calculated,
                     kt, m_global))
        
        # learning rate decay
        if (it + 1) % 2000 == 0:
            lr_i *= 0.95
        
        # save
        if (it + 1) % 1000 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 100 == 0:
            f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample})

            save_dir = './BEGAN/sample_images_while_training/run%s' % run_num
            mkdir(save_dir + '/')
            imwrite(immerge(f_sample_opt, 10, 10), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))

except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

 [*] Loading checkpoint...
 [*] No suitable checkpoint!
Epoch: (  0) (    1/ 1208) -- g_loss: 0.7356, d_loss: 0.6701 AE_r: 0.6701, AE_f: 0.7356, kt: 0.00000000, M: 1.07066715
Epoch: (  0) (  101/ 1208) -- g_loss: 0.1442, d_loss: 0.2052 AE_r: 0.2052, AE_f: 0.1442, kt: 0.00000000, M: 0.24682634
Epoch: (  0) (  201/ 1208) -- g_loss: 0.1058, d_loss: 0.1794 AE_r: 0.1794, AE_f: 0.1058, kt: 0.00000000, M: 0.19556177
Epoch: (  0) (  301/ 1208) -- g_loss: 0.0778, d_loss: 0.1651 AE_r: 0.1651, AE_f: 0.0778, kt: 0.00000545, M: 0.16981589
Epoch: (  0) (  401/ 1208) -- g_loss: 0.0760, d_loss: 0.1574 AE_r: 0.1574, AE_f: 0.0760, kt: 0.00002277, M: 0.16007054
Epoch: (  0) (  501/ 1208) -- g_loss: 0.0720, d_loss: 0.1534 AE_r: 0.1534, AE_f: 0.0720, kt: 0.00026636, M: 0.15809718
Epoch: (  0) (  601/ 1208) -- g_loss: 0.0761, d_loss: 0.1595 AE_r: 0.1595, AE_f: 0.0761, kt: 0.00095698, M: 0.16317688
Epoch: (  0) (  701/ 1208) -- g_loss: 0.0839, d_loss: 0.1572 AE_r: 0.1573, AE_f: 0.0839, kt: 0.00181920, M: 0.1

Epoch: (  5) (  461/ 1208) -- g_loss: 0.0568, d_loss: 0.1168 AE_r: 0.1175, AE_f: 0.0568, kt: 0.01381732, M: 0.11948251
Epoch: (  5) (  561/ 1208) -- g_loss: 0.0646, d_loss: 0.1198 AE_r: 0.1207, AE_f: 0.0646, kt: 0.01363286, M: 0.12489434
Epoch: (  5) (  661/ 1208) -- g_loss: 0.0750, d_loss: 0.1099 AE_r: 0.1108, AE_f: 0.0750, kt: 0.01336300, M: 0.13043889
Epoch: (  5) (  761/ 1208) -- g_loss: 0.0595, d_loss: 0.1163 AE_r: 0.1170, AE_f: 0.0595, kt: 0.01316805, M: 0.11798152
Epoch: (  5) (  861/ 1208) -- g_loss: 0.0647, d_loss: 0.1048 AE_r: 0.1056, AE_f: 0.0647, kt: 0.01309334, M: 0.11746248
Model saved in file: ./BEGAN/checkpoints/run9/Epoch_(5)_(960of1208).ckpt
Epoch: (  5) (  961/ 1208) -- g_loss: 0.0625, d_loss: 0.1197 AE_r: 0.1204, AE_f: 0.0625, kt: 0.01305000, M: 0.12276014
Epoch: (  5) ( 1061/ 1208) -- g_loss: 0.0601, d_loss: 0.1097 AE_r: 0.1105, AE_f: 0.0601, kt: 0.01296868, M: 0.11537825
Epoch: (  5) ( 1161/ 1208) -- g_loss: 0.0576, d_loss: 0.1038 AE_r: 0.1045, AE_f: 0.0576, kt: 0

Epoch: ( 10) (  921/ 1208) -- g_loss: 0.0530, d_loss: 0.1095 AE_r: 0.1100, AE_f: 0.0530, kt: 0.00948696, M: 0.11200358
Epoch: ( 10) ( 1021/ 1208) -- g_loss: 0.0609, d_loss: 0.1065 AE_r: 0.1071, AE_f: 0.0609, kt: 0.00933726, M: 0.11446221
Epoch: ( 10) ( 1121/ 1208) -- g_loss: 0.0562, d_loss: 0.1068 AE_r: 0.1073, AE_f: 0.0562, kt: 0.00917866, M: 0.10987800
Epoch: ( 11) (   13/ 1208) -- g_loss: 0.0632, d_loss: 0.1074 AE_r: 0.1079, AE_f: 0.0632, kt: 0.00897975, M: 0.11712861
Epoch: ( 11) (  113/ 1208) -- g_loss: 0.0599, d_loss: 0.1099 AE_r: 0.1104, AE_f: 0.0599, kt: 0.00888414, M: 0.11511378
Epoch: ( 11) (  213/ 1208) -- g_loss: 0.0555, d_loss: 0.1105 AE_r: 0.1110, AE_f: 0.0555, kt: 0.00864284, M: 0.11100149
Epoch: ( 11) (  313/ 1208) -- g_loss: 0.0538, d_loss: 0.1106 AE_r: 0.1110, AE_f: 0.0538, kt: 0.00856094, M: 0.11272662
Epoch: ( 11) (  413/ 1208) -- g_loss: 0.0602, d_loss: 0.1037 AE_r: 0.1042, AE_f: 0.0602, kt: 0.00836671, M: 0.11233125
Epoch: ( 11) (  513/ 1208) -- g_loss: 0.0595, d_

Epoch: ( 16) (  273/ 1208) -- g_loss: 0.0547, d_loss: 0.1078 AE_r: 0.1081, AE_f: 0.0547, kt: 0.00543232, M: 0.10876576
Epoch: ( 16) (  373/ 1208) -- g_loss: 0.0543, d_loss: 0.1113 AE_r: 0.1115, AE_f: 0.0543, kt: 0.00520339, M: 0.11295637
Epoch: ( 16) (  473/ 1208) -- g_loss: 0.0583, d_loss: 0.1023 AE_r: 0.1026, AE_f: 0.0583, kt: 0.00502361, M: 0.10954330
Epoch: ( 16) (  573/ 1208) -- g_loss: 0.0551, d_loss: 0.1103 AE_r: 0.1106, AE_f: 0.0551, kt: 0.00472448, M: 0.11076194
Model saved in file: ./BEGAN/checkpoints/run9/Epoch_(16)_(672of1208).ckpt
Epoch: ( 16) (  673/ 1208) -- g_loss: 0.0535, d_loss: 0.1007 AE_r: 0.1009, AE_f: 0.0535, kt: 0.00444984, M: 0.10392333
Epoch: ( 16) (  773/ 1208) -- g_loss: 0.0557, d_loss: 0.1032 AE_r: 0.1035, AE_f: 0.0557, kt: 0.00426398, M: 0.10740943
Epoch: ( 16) (  873/ 1208) -- g_loss: 0.0513, d_loss: 0.1029 AE_r: 0.1031, AE_f: 0.0513, kt: 0.00409791, M: 0.10346049
Epoch: ( 16) (  973/ 1208) -- g_loss: 0.0518, d_loss: 0.1048 AE_r: 0.1050, AE_f: 0.0518, kt: 

Epoch: ( 21) (  733/ 1208) -- g_loss: 0.0508, d_loss: 0.1087 AE_r: 0.1090, AE_f: 0.0508, kt: 0.00419077, M: 0.11265823
Epoch: ( 21) (  833/ 1208) -- g_loss: 0.0553, d_loss: 0.1024 AE_r: 0.1026, AE_f: 0.0553, kt: 0.00414052, M: 0.10657070
Epoch: ( 21) (  933/ 1208) -- g_loss: 0.0488, d_loss: 0.1060 AE_r: 0.1062, AE_f: 0.0488, kt: 0.00429953, M: 0.11042653
Epoch: ( 21) ( 1033/ 1208) -- g_loss: 0.0540, d_loss: 0.1020 AE_r: 0.1022, AE_f: 0.0540, kt: 0.00460392, M: 0.10506559
Epoch: ( 21) ( 1133/ 1208) -- g_loss: 0.0459, d_loss: 0.1021 AE_r: 0.1023, AE_f: 0.0459, kt: 0.00485342, M: 0.10754471
Epoch: ( 22) (   25/ 1208) -- g_loss: 0.0504, d_loss: 0.0963 AE_r: 0.0965, AE_f: 0.0504, kt: 0.00490580, M: 0.09864666
Epoch: ( 22) (  125/ 1208) -- g_loss: 0.0510, d_loss: 0.0973 AE_r: 0.0975, AE_f: 0.0510, kt: 0.00490580, M: 0.09980148
Epoch: ( 22) (  225/ 1208) -- g_loss: 0.0504, d_loss: 0.1026 AE_r: 0.1028, AE_f: 0.0504, kt: 0.00485028, M: 0.10383901
Epoch: ( 22) (  325/ 1208) -- g_loss: 0.0511, d_

Epoch: ( 27) (   85/ 1208) -- g_loss: 0.0526, d_loss: 0.0966 AE_r: 0.0969, AE_f: 0.0526, kt: 0.00486580, M: 0.10099107
Epoch: ( 27) (  185/ 1208) -- g_loss: 0.0504, d_loss: 0.0967 AE_r: 0.0970, AE_f: 0.0504, kt: 0.00498398, M: 0.09891035
 [*] Close main session!


KeyboardInterrupt: 

### Using a trained network

In [8]:
sample_num = 0

In [14]:
""" test """

run_num = 6
epoch = 91
it_epoch = 872
batch_epoch = 1208

''' init '''
#Directory to save sample images from generator in.
test_sample_directory = './BEGAN/test_images_after_training/run%s/Epoch_(%d)_(%dof%d)' % (run_num, epoch, it_epoch, batch_epoch)

#Directory to load trained model from. : run_num, epoch, it_epoch, batch_epoch
ckpt_dir = './BEGAN/checkpoints/run%s' % run_num
batch_size_sample = 100

# session
sess = session()

''' Reload the model '''
load_checkpoint(ckpt_dir, sess)
print('Loading Model...')

''' Generate '''
try:
    #z_ipt_sample = np.random.normal(size=[100, z_dim])
    z_ipt_sample = np.random.uniform(-1., 1., size=[batch_size_sample, z_dim]) #Generate a random z batch
    
    f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(test_sample_directory):
        os.makedirs(test_sample_directory)
    
    imwrite(immerge(f_sample_opt, 10, 10), test_sample_directory+'/fig'+str(sample_num)+'.jpg')
    print(test_sample_directory+'/fig'+str(sample_num)+'.jpg')
    sample_num += 1
    
except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

 [*] Loading checkpoint...
INFO:tensorflow:Restoring parameters from ./BEGAN/checkpoints/run6/Epoch_(91)_(72of1208).ckpt
 [*] Loading successful! Copy variables from ./BEGAN/checkpoints/run6/Epoch_(91)_(72of1208).ckpt
Loading Model...
./BEGAN/test_images_after_training/run6/Epoch_(91)_(872of1208)/fig2.jpg
 [*] Close main session!
